<a href="https://colab.research.google.com/github/Joaoplims/NLP-Projects/blob/main/TP1_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install gensim scikit-learn numpy

import gensim
import requests
import urllib.request
import zipfile
import gensim.downloader as api
import requests
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from io import StringIO


# Baixar o corpus text8
corpus = api.load('text8')

# Converter o corpus para uma lista de listas de palavras (sentenças)
text8Data = [d for d in corpus]


# Função para treinar o modelo
def train_model(corpus, size, window, sg, epochs):
    model = Word2Vec(corpus, vector_size=size, window=window, sg=sg, epochs=epochs)
    model.save(f'model_size{size}_window{window}_sg{sg}_epochs{epochs}.model')
    return model

# Treinar diversos modelos
#model1 = train_model(text8Data, size=100, window=5, sg=0, epochs=5)  # CBOW
#model2 = train_model(text8Data, size=200, window=10, sg=1, epochs=10) # Skip-gram

# Carregar o modelo salvo
model1 = Word2Vec.load('/content/model_size100_window5_sg0_epochs5.model')
model1 = Word2Vec.load('/content/model_size200_window10_sg1_epochs10.model')



# Função para avaliar o modelo
def evaluate_analogy(analogy_file, model):
    with open(analogy_file, 'r') as file:
        lines = file.readlines()

    distances = []
    for line in lines:
        words = line.split()
        if len(words) != 4:
            continue

        # Check if all words are in the vocabulary before proceeding
        if all(word in model.wv for word in words):
            vec = model.wv[words[1]] + model.wv[words[0]] - model.wv[words[2]]
            expected = model.wv[words[3]]
            distance = cosine_similarity([vec], [expected])[0][0]
            distances.append(distance)
        #else:
            # Handle cases where words are not in the vocabulary (e.g., skip or print a message)
         #   print(f"Skipping analogy: {line.strip()} - One or more words not in vocabulary")

    # Return 0 if no analogies were evaluated to avoid division by zero
    if not distances:
        return 0

    return sum(distances) / len(distances)

# Avaliar o modelo carregado
avg_distance1 = evaluate_analogy('/content/questions-words.txt', model1)
print(f'Média das distâncias do cosseno Modelo1: {avg_distance1}')

print("----------- FIM ANALOGIA MODELO modelo 1 -----------");




# Avaliar o modelo
avg_distance2 = evaluate_analogy('/content/questions-words.txt', model2)
print(f'Média das distâncias do cosseno para o Modelo2: {avg_distance2}')

print("----------- FIM ANALOGIA MODELO modelo 2 -----------");

Média das distâncias do cosseno Modelo1: -0.004335553103754178
----------- FIM ANALOGIA MODELO modelo 1 -----------
Média das distâncias do cosseno para o Modelo2: -0.004335553103754178
----------- FIM ANALOGIA MODELO modelo 2 -----------
